In [4]:

from __future__ import division
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import time
import argparse
import torch
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import torch.nn.functional as F
from utils import accuracy, AverageMeter, save_checkpoint, visualize_graph, get_parameters_size
from torch.utils.tensorboard import SummaryWriter


import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt

## torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

# dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn import DataParallel
import tqdm


# read image
import PIL
from PIL import Image
from utils import *

ms_polyu_path = 'dataset/MS_PolyU/'
casia_path = 'dataset/CASIA-Multi-Spectral-PalmprintV1/images/'

r_img_path = ms_polyu_path + 'Red_ind/'
b_img_path =  ms_polyu_path + 'Blue_ind/'
n_img_path =  ms_polyu_path + 'NIR_ind/'
g_img_path =  ms_polyu_path + 'Green_ind/'

################ DATASET CLASS
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 
# one_hot_embedding(1, 10)
def part_init(istrain=True):
    r_list = []
    b_list = []
    vein_list = []
    prints_list = []
    labels = []
    
        # split all data into train, test data
    train_ratio = 1
    train_num = int(500 * train_ratio)
    print("split train users:",train_num)
    if istrain:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(8):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
                
                g_img = np.array(Image.open(os.path.join(g_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                g_normed = (g_img - g_img.min()) / (g_img.max()-g_img.min())

                b_img = np.array(Image.open(os.path.join(b_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

                n_img = np.array(Image.open(os.path.join(n_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
                rb = r_normed - b_normed * 0.5
                rb =  (rb * 128+128).astype(np.uint8)

                imgprint = np.dstack((r_img,g_img,b_img))
                imgvein = np.dstack((rb, n_img))
                
                vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)
    else:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(8,12):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
                
                g_img = np.array(Image.open(os.path.join(g_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                g_normed = (g_img - g_img.min()) / (g_img.max()-g_img.min())

                b_img = np.array(Image.open(os.path.join(b_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

                n_img = np.array(Image.open(os.path.join(n_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
                rb = r_normed - b_normed * 0.5
                rb =  (rb * 128+128).astype(np.uint8)
                imgprint = np.dstack((r_img,g_img,b_img))
                imgvein = np.dstack((rb, n_img))
                
                vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)



    # return np.array(r_list), np.array(b_list), np.array(n_list), np.array(labels),np.array(r_list_test), np.array(b_list_test), np.array(n_list_test), np.array(labels_test)
    return  vein_list,prints_list, labels

# r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
class load_data(Dataset):
    """Loads the Data."""
    def __init__(self, training=True):

        self.training = training
#         r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
        self.transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ColorJitter(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomPerspective(),
        transforms.RandomAffine(30),
        transforms.ToTensor(),
#         transforms.Resize((224, 224)),# if resnet
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        self.transform_test = transforms.Compose([
        transforms.ToPILImage(),
#         transforms.Resize((224, 224)),# if resnet
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        if self.training:
            print('\n...... Train files loading\n')
            self.vein_list,self.prints_list, self.labels= part_init(istrain=True)
            print('\nTrain files loaded ......\n')
        else:
            print('\n...... Test files loading\n')
            self.vein_list,self.prints_list, self.labels = part_init(istrain=False)
            print('\nTest files loaded ......\n')

    def __len__(self):
        return len(self.vein_list)

         
    def __getitem__(self, idx):

        if self.training:
            prints_img = self.transform(self.prints_list[idx])
            vein_img = self.transform(self.vein_list[idx])
        else:
            prints_img = self.transform_test(self.prints_list[idx])
            vein_img = self.transform_test(self.vein_list[idx])
        
        label = self.labels[idx]
        
        n_img = np.dstack((prints_img[0,:,:],prints_img[1,:,:],prints_img[2,:,:],vein_img[0,:,:],vein_img[1,:,:]))
        
        return n_img,label


In [5]:
# train_loader, test_loader, dataset_loader, num_train, num_test, num_dataset = get_data(config)
batch_size = 256
# train_loader = DataLoader(load_data(training=True), batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True,prefetch_factor=8)  # ,prefetch_factor=2
test_loader = DataLoader(load_data(training=False), batch_size=64, shuffle=False)  # ,prefetch_factor=2



...... Test files loading

split train users: 500


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:48<00:00, 10.37it/s]


Test files loaded ......



In [10]:
for _, img in tqdm.tqdm(enumerate(test_loader)):
    print((img[0].size(), img[1].size()))
    break

0it [00:00, ?it/s]

(torch.Size([64, 1, 128, 128]), torch.Size([64, 500]))


In [8]:
import numpy as np
import os
# import cv2
from  matplotlib import pyplot as plt

## torch
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchsummary import summary

# dataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn import DataParallel
import tqdm


# read image
import PIL
from PIL import Image
from utils import *

ms_polyu_path = 'dataset/MS_PolyU/'
casia_path = 'dataset/CASIA-Multi-Spectral-PalmprintV1/images/'

r_img_path = ms_polyu_path + 'Red_ind/'
b_img_path =  ms_polyu_path + 'Blue_ind/'
n_img_path =  ms_polyu_path + 'NIR_ind/'
g_img_path =  ms_polyu_path + 'Green_ind/'

################ DATASET CLASS
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 
# one_hot_embedding(1, 10)
def part_init(istrain=True):
    r_list = []
    b_list = []
    vein_list = []
    prints_list = []
    labels = []
    
        # split all data into train, test data
    train_ratio = 1
    train_num = int(500 * train_ratio)
    print("split train users:",train_num)
    if istrain:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(8):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
                
#                 g_img = np.array(Image.open(os.path.join(g_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 g_normed = (g_img - g_img.min()) / (g_img.max()-g_img.min())

#                 b_img = np.array(Image.open(os.path.join(b_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

#                 n_img = np.array(Image.open(os.path.join(n_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 r_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
#                 rb = r_normed - b_normed * 0.5
#                 rb =  (rb * 128+128).astype(np.uint8)

                imgprint = r_img
#                 imgvein = np.dstack((rb, n_img))
                
#                 vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)
    else:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(8,12):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
                
#                 g_img = np.array(Image.open(os.path.join(g_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 g_normed = (g_img - g_img.min()) / (g_img.max()-g_img.min())

#                 b_img = np.array(Image.open(os.path.join(b_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

#                 n_img = np.array(Image.open(os.path.join(n_img_path, "%04d_"%(i+1)+"%04d.jpg"%(j+1))))
#                 r_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
#                 rb = r_normed - b_normed * 0.5
#                 rb =  (rb * 128+128).astype(np.uint8)
                imgprint = r_img
#                 imgvein = np.dstack((rb, n_img))
                
#                 vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)



    # return np.array(r_list), np.array(b_list), np.array(n_list), np.array(labels),np.array(r_list_test), np.array(b_list_test), np.array(n_list_test), np.array(labels_test)
    return  vein_list,prints_list, labels

# r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
class load_data(Dataset):
    """Loads the Data."""
    def __init__(self, training=True):

        self.training = training
#         r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
        self.transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ColorJitter(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomPerspective(),
        transforms.RandomAffine(30),
        transforms.ToTensor(),
#         transforms.Resize((224, 224)),# if resnet
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        self.transform_test = transforms.Compose([
        transforms.ToPILImage(),
#         transforms.Resize((224, 224)),# if resnet
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        if self.training:
            print('\n...... Train files loading\n')
            self.vein_list,self.prints_list, self.labels= part_init(istrain=True)
            print('\nTrain files loaded ......\n')
        else:
            print('\n...... Test files loading\n')
            self.vein_list,self.prints_list, self.labels = part_init(istrain=False)
            print('\nTest files loaded ......\n')

    def __len__(self):
        return len(self.prints_list)

         
    def __getitem__(self, idx):

        if self.training:
            prints_img = self.transform(self.prints_list[idx])
#             vein_img = self.transform(self.vein_list[idx])
        else:
            prints_img = self.transform_test(self.prints_list[idx])
#             vein_img = self.transform_test(self.vein_list[idx])
        
        label = self.labels[idx]
        
        n_img = prints_img
        
        return n_img,label


In [9]:
# train_loader, test_loader, dataset_loader, num_train, num_test, num_dataset = get_data(config)
batch_size = 256
# train_loader = DataLoader(load_data(training=True), batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True,prefetch_factor=8)  # ,prefetch_factor=2
test_loader = DataLoader(load_data(training=False), batch_size=64, shuffle=False)  # ,prefetch_factor=2



...... Test files loading

split train users: 500


100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 324.47it/s]


Test files loaded ......



XXX_(L/R) _ YYY_ZZ .jpg 

XXX: the unique identifier of people, ranges from 000 to 100. 

(L/R): the type of palm, ‘L' denotes left palm and ‘R' denotes right palm 

YYY: electromagnetic spectrums."WHT" represents the white light. 

ZZ: the index of samples ranging from 01 to 06. 01 to 03 belong to the first session. 04 to 06 
belong to the second session.

In [27]:
# R	B	N
# 700	460	850

casia_path = 'dataset/Casia_M/'

r_img_path = casia_path + 'casia_700/'
b_img_path =  casia_path + 'casia_460/'
n_img_path =  casia_path + 'casia_850/'
g_img_path =  casia_path + 'casia_850/'

################ DATASET CLASS
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 
# one_hot_embedding(1, 10)
def part_init(istrain=True):
    r_list = []
    b_list = []
    vein_list = []
    prints_list = []
    labels = []
    
        # split all data into train, test data
    train_ratio = 1
    train_num = int(200 * train_ratio)# 200 users
    print("split train users:",train_num)
    if istrain:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(4):
                # XXX_(L/R) _ YYY_ZZ .jpg
                r_img = np.array(Image.open(os.path.join(r_img_path, "%03d_"%(i+1)+"%02d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
         
                b_img = np.array(Image.open(os.path.join(b_img_path, "%03d_"%(i+1)+"%02d.jpg"%(j+1))))
                b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

                n_img = np.array(Image.open(os.path.join(n_img_path, "%03d_"%(i+1)+"%02d.jpg"%(j+1))))
                # n_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
                rb = r_normed - b_normed * 0.5
                rb =  (rb * 128+128).astype(np.uint8)

                imgprint = np.dstack((r_img,b_img,b_img))
                imgvein = np.dstack((rb, n_img))
                
                vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)
    else:
        for i in tqdm.tqdm(range(train_num)):
            for j in range(4,6):
                r_img = np.array(Image.open(os.path.join(r_img_path, "%03d_"%(i+1)+"%02d.jpg"%(j+1))))
                r_normed = (r_img - r_img.min()) / (r_img.max()-r_img.min())
         
                b_img = np.array(Image.open(os.path.join(b_img_path, "%03d_"%(i+1)+"%02d.jpg"%(j+1))))
                b_normed = (b_img - b_img.min()) / (b_img.max()-b_img.min())

                n_img = np.array(Image.open(os.path.join(n_img_path, "%03d_"%(i+1)+"%02d.jpg"%(j+1))))
                # n_normed = (n_img - n_img.min()) / (n_img.max()-n_img.min())
                
                rb = r_normed - b_normed * 0.5
                rb =  (rb * 128+128).astype(np.uint8)

                imgprint = np.dstack((r_img,b_img,b_img))
                imgvein = np.dstack((rb, n_img))
                
                vein_list.append(imgvein)
                prints_list.append(imgprint)
                labels.append(one_hot_embedding(i, train_num))
#                 labels.append(i)



    # return np.array(r_list), np.array(b_list), np.array(n_list), np.array(labels),np.array(r_list_test), np.array(b_list_test), np.array(n_list_test), np.array(labels_test)
    return  vein_list,prints_list, labels

# r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
class load_data(Dataset):
    """Loads the Data."""
    def __init__(self, training=True):

        self.training = training
#         r_list, b_list, n_list, labels,r_list_test, b_list_test, n_list_test, labels_test = part_init()
        self.transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),# if casia
        transforms.ColorJitter(),
        transforms.RandomRotation(degrees=15),
        transforms.RandomPerspective(),
        transforms.RandomAffine(30),
        transforms.ToTensor(),
#         transforms.Resize((224, 224)),# if resnet
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        self.transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),# if casia
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225]),
    ])
        if self.training:
            print('\n...... Train files loading\n')
            self.vein_list,self.prints_list, self.labels= part_init(istrain=True)
            print('\nTrain files loaded ......\n')
        else:
            print('\n...... Test files loading\n')
            self.vein_list,self.prints_list, self.labels = part_init(istrain=False)
            print('\nTest files loaded ......\n')

    def __len__(self):
        return len(self.vein_list)

         
    def __getitem__(self, idx):

        if self.training:
            prints_img = self.transform(self.prints_list[idx])
            vein_img = self.transform(self.vein_list[idx])
        else:
            prints_img = self.transform_test(self.prints_list[idx])
            vein_img = self.transform_test(self.vein_list[idx])
        
        label = self.labels[idx]
        
        n_img = np.dstack((prints_img[0,:,:],prints_img[1,:,:],prints_img[2,:,:],vein_img[0,:,:],vein_img[1,:,:]))
        
        return n_img,label


In [34]:
# train_loader, test_loader, dataset_loader, num_train, num_test, num_dataset = get_data(config)
batch_size = 8
train_loader = DataLoader(load_data(training=True), batch_size=batch_size, shuffle=True)  # ,prefetch_factor=2
test_loader = DataLoader(load_data(training=False), batch_size=64, shuffle=False)  # ,prefetch_factor=2



...... Train files loading

split train users: 200


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 77.79it/s]



Train files loaded ......


...... Test files loading

split train users: 200


100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 126.26it/s]


Test files loaded ......



In [35]:
for _, img in tqdm.tqdm(enumerate(train_loader)):
    print((img[0].size(), img[1].size()))
    break
# for _, img in tqdm.tqdm(enumerate(test_loader)):
#     print((img[0].size(), img[1].size()))
#     break

0it [00:00, ?it/s]

(torch.Size([8, 128, 128, 5]), torch.Size([8, 200]))
